# Baxter Introduction

Learn how to move the baxter in joint space (forward kinematics) and how to operate its two finger and vacuum grippers.

##### ENGG 4460 University of Guelph, Patrick Wspanialy

Before you start working with Baxter and this notebook, make sure you launched the notebook from a `Baxter shell`.

You can start a Baxter shell by entering the following command into a `Terminal`.

    bash --rcfile ~/.bashrc.baxter 
    
Then close this window and rerun `ipython notebook` in the new `Baxter shell` you just created.

Import python modules needed to control Baxter

In [ ]:
import pprint
import math
import rospy
import baxter_interface
from baxter_interface import RobotEnable
from baxter_interface import Gripper
pp = pprint.PrettyPrinter(indent=4)

Initialize a new ROS node and name it "Hello_Baxter", 

In [ ]:
rospy.init_node('Hello_Baxter')

If you didn't run the `rosrun baxter_tools enable_robot.py -e` command, you can also enable baxter directly from Python.

In [ ]:
baxter = RobotEnable()
baxter.enable()

create a new limb objects for each arm, and print out the current limb joint angles

In [ ]:
right_arm = baxter_interface.Limb('right')
left_arm = baxter_interface.Limb('left')
right_angles = right_arm.joint_angles()
left_angles = left_arm.joint_angles()
pp.pprint(right_angles)
pp.pprint(left_angles)

Set new angles for the 7 joints of the arm and print them

In [ ]:
right_angles['right_s0']=0.0
right_angles['right_s1']=0.0
right_angles['right_e0']=0.0
right_angles['right_e1']=0.0
right_angles['right_w0']=0.0
right_angles['right_w1']=0.0
right_angles['right_w2']=0.0

pp.pprint(right_angles)

[<img src="http://sdk.rethinkrobotics.com/mediawiki-1.22.2/images/7/7c/Baxter_arm.png">]

Move the right arm to the set joint angles (this is forward kinematics)

In [ ]:
right_arm.move_to_joint_positions(right_angles)

Get the Cartesian space coordinates of the right hand

In [ ]:
right_pose = right_arm.endpoint_pose()

pp.pprint(right_pose['position'])
print "\n"
pp.pprint(right_pose['orientation'])

Repeat for process for the other arm

In [ ]:
left_angles['left_s0']=0.0
left_angles['left_s1']=0.0
left_angles['left_e0']=0.0
left_angles['left_e1']=0.0
left_angles['left_w0']=0.0
left_angles['left_w1']=0.0
left_angles['left_w2']=0.0

pp.pprint(left_angles)


left_arm.move_to_joint_positions(left_angles)

Make two new sets of joint angles for the right arm, corresponding to the beginning and end of a waving motion

In [ ]:
wave_1 = {'right_s0': -0.459, 'right_s1': -0.202, 'right_e0': 1.807, 'right_e1': 1.714, 'right_w0': -0.906, 'right_w1': -1.545, 'right_w2': -0.276}
wave_2 = {'right_s0': -0.395, 'right_s1': -0.202, 'right_e0': 1.831, 'right_e1': 1.981, 'right_w0': -1.979, 'right_w1': -1.100, 'right_w2': -0.448}

Using a `for loop`, repeat the wave twice

In [ ]:
for _move in range(2):
    right_arm.move_to_joint_positions(wave_1)
    right_arm.move_to_joint_positions(wave_2)

Initialize Baxter's end effectors. It has a two finger gripper on its right arm, and a vacuum on its left arm.

In [ ]:
fingers = Gripper('right')
fingers.calibrate()

vacuum = Gripper('left')
vacuum_sensor = baxter_interface.AnalogIO('left_vacuum_sensor_analog')

Close the two finger gripper

In [ ]:
fingers.close()

Open the two finger gripper

In [ ]:
fingers.open()

Turn on the vacuum and attempt to pick up an object. You can set a `timeout`, which will stop the vacuum if an object hasn't been detected. The default `timeout` value is 5 seconds.

In [ ]:
vacuum.close(timeout=6)

Turn off the vacuum and release what it's holding.

In [ ]:
vacuum.open()

Check if the vacuum gipper is holding anything.

In [ ]:
vacuum_strength = vacuum_sensor.state()
if vacuum_strength > 50:
   print "I've got it! :)  [vacuum strength: " + str(vacuum_strength) + "]"
else:
    print "I'm not holding anything :(  [vacuum strength: " + str(vacuum_strength) + "]"

Disable baxter once you're done using him. Make sure to guide his arms into the correct resting position before disabling.

This example also shows you how you can define angle positions using degrees, and let python convert it to radians for you using `math.radians()`. You can go the other way when printing out the current position using `math.degrees()`.

In [ ]:
right_angles['right_s0'] = math.radians(20)
right_angles['right_s1'] = 0.0
right_angles['right_e0'] = 0.0
right_angles['right_e1'] = 0.0
right_angles['right_w0'] = 0.0
right_angles['right_w1'] = 0.0
right_angles['right_w2'] = 0.0

left_angles['left_s0'] = math.radians(-20)
left_angles['left_s1'] = 0.0
left_angles['left_e0'] = 0.0
left_angles['left_e1'] = 0.0
left_angles['left_w0'] = 0.0
left_angles['left_w1'] = 0.0
left_angles['left_w2'] = 0.0

right_arm.move_to_joint_positions(right_angles)
left_arm.move_to_joint_positions(left_angles)

In [ ]:
baxter.disable()